## Motivation

Understanding the results of the STP is not straightforward and requires a grasp of the module equation. To simplify this, I will attempt to demonstrate the results using basic operations, eliminating the need to read the module equation.


I will use the example to show.

ASSERT( Q_13 = 0x3C );ASSERT( Q_4 = 0x33 );ASSERT( X_2 = 0x55 );ASSERT( T_5 = 0x77 );ASSERT( Q_0 = 0x0F );ASSERT( Q_1 = 0x33 );ASSERT( Q_2 = 0x55 );ASSERT( B_2 = 0x04 );ASSERT( Q_5 = 0x3C );ASSERT( Q_6 = 0x55 );ASSERT( Q_7 = 0x30 );ASSERT(Q_8 = 0x05);
ASSERT( Q_12 = 0x77 );ASSERT( Q_3 = 0x0F );ASSERT( X_0 = 0x0F );ASSERT( T_2 = 0x30 );ASSERT( T_1 = 0x05 );ASSERT( B_0 = 0x02 );ASSERT( Y_0 = 0x4B );ASSERT( X_1 = 0x33 );
ASSERT( B_5 = 0x06 );ASSERT( Q_10 = 0x55 );ASSERT( Y_1 = 0x39 );ASSERT( B_6 = 0x02 );
ASSERT( B_4 = 0x02 );ASSERT( T_6 = 0x4B );ASSERT( B_3 = 0x02 );ASSERT( T_4 = 0x39 );ASSERT( B_1 = 0x04 );ASSERT( T_3 = 0x65 );ASSERT( T_0 = 0x3C );ASSERT( Q_9 = 0x3C );
ASSERT( Q_11 = 0x33 );ASSERT( Y_2 = 0x65 );
Invalid.
081.35557174682617


In [14]:
# extract target variables from a result file
result_file_str = "result.txt"
lines = None

# Open the file
with open(result_file_str, "r") as result_file:
    lines = result_file.readlines()

# Initialize an empty dictionary
variables = {}

for line in lines:
    # Only process lines that start with 'ASSERT'
    if line.startswith("ASSERT"):
        # Remove the 'ASSERT( ' and ' );' parts from the line
        line = line.replace("ASSERT( ", "").replace(" );", "")

        # Split the line at the '=' sign
        var, value = line.split(" = ")

        # Remove the newline character from the value
        value = value.strip()

        # Add the variable and its value to the dictionary
        variables[var] = value

print(variables)

{'Q_13': '0x3C', 'Q_4': '0x33', 'X_2': '0x55', 'T_5': '0x77', 'Q_0': '0x0F', 'Q_1': '0x33', 'Q_2': '0x55', 'B_2': '0x04', 'Q_5': '0x3C', 'Q_6': '0x55', 'Q_7': '0x30', 'Q_8': '0x05', 'Q_12': '0x77', 'Q_3': '0x0F', 'X_0': '0x0F', 'T_2': '0x30', 'T_1': '0x05', 'B_0': '0x02', 'Y_0': '0x4B', 'X_1': '0x33', 'B_5': '0x06', 'Q_10': '0x55', 'Y_1': '0x39', 'B_6': '0x02', 'B_4': '0x02', 'T_6': '0x4B', 'B_3': '0x02', 'T_4': '0x39', 'B_1': '0x04', 'T_3': '0x65', 'T_0': '0x3C', 'Q_9': '0x3C', 'Q_11': '0x33', 'Y_2': '0x65'}


In [15]:
import re

# Initialize empty dictionaries
arrays = {}

# Iterate over each item in the dictionary
for key, value in variables.items():
    # Get the first character of the key
    first_char = key[0]

    # Get the last number in the key using regex
    match = re.search(r"\d+$", key)
    if match:
        last_number = int(match.group())

        # If this character doesn't have an array yet, create it
        if first_char not in arrays:
            arrays[first_char] = {}

        # Add the value to the appropriate array at the position specified by the last number
        arrays[first_char][last_number] = value

# Sort each array by its keys (the last numbers in the original keys)
for key in arrays:
    arrays[key] = [item[1] for item in sorted(arrays[key].items())]

print(arrays)

{'Q': ['0x0F', '0x33', '0x55', '0x0F', '0x33', '0x3C', '0x55', '0x30', '0x05', '0x3C', '0x55', '0x33', '0x77', '0x3C'], 'X': ['0x0F', '0x33', '0x55'], 'T': ['0x3C', '0x05', '0x30', '0x65', '0x39', '0x77', '0x4B'], 'B': ['0x02', '0x04', '0x04', '0x02', '0x02', '0x06', '0x02'], 'Y': ['0x4B', '0x39', '0x65']}


In [25]:
X = list(arrays["X"])
Y = list(arrays["Y"])
B = list(arrays["B"])
T = list(arrays["T"])
Q = list(arrays["Q"])

BT = {
    "0x02": "XOR",
    "0x04": "AND",
    "0x06": "OR",
    "0x03": "NOT1",
    "0x01": "NOT0",
}

XS = [f"X{i}" for i in range(0, len(X))]
BS = [BT[b] for b in B]

TS = []
for i in range(0, len(T)):
    q1 = Q[i * 2]
    q2 = Q[i * 2 + 1]
    q1_s = ""
    q2_s = ""
    if q1 in X:
        q1_s = XS[X.index(q1)]
    if q2 in X:
        q2_s = XS[X.index(q2)]
    if q1 in T:
        q1_s = TS[T.index(q1)]
    if q2 in T:
        q2_s = TS[T.index(q2)]
    TS.append(f"{BS[i]} ({q1_s}, {q2_s}) ")

YS = []
for i in range(0, len(Y)):
    y = Y[i]
    if y in T:
        YS.append(f"Y{i} = {TS[T.index(y)]}")

print(YS)

['Y0 = XOR (OR (X2, X1) , XOR (X0, X1) ) ', 'Y1 = XOR (AND (X2, X0) , XOR (X0, X1) ) ', 'Y2 = XOR (X2, AND (X1, XOR (X0, X1) ) ) ']


In [27]:
y2 = 0x65 
x0 = 0x0F
x1 = 0x33
x2 = 0x55
if y2 == x2 ^ (x1 & (x0 ^ x1)):
    print("Correct")

Correct
